NOTEBOOK TO GENERATE NEW CONFIGURATIONS

In [1]:
import numpy as np
from configurator import Configurator, MAXIMUMS_QUAT, MAXIMUMS_EULER

Default Data Configuration

In [2]:
# Data Generation, Does not need to be ran every time
constraints = {
    "lat": (-10, 10),
    "lon": (-10, 10),
    "alt": (300, 12000),
    "phi": ((-np.pi / 32), (np.pi / 32)),  # Radians?
    "theta": ((-np.pi / 16), (np.pi / 16)),  # Radians?
    "psi": ((-np.pi / 32), (np.pi / 32)),  # Radians?
    "v_x": (12.5, 60),  # m/s
    "v_y": (-10, 10),  # m/s
    "v_z": (-20, 20),  # m/s
    "P": (-10, 10),  # Rad / S
    "Q": (-10, 10),  # Rad / S
    "R": (-10, 10),  # Rad / S
}

freq = 0.01  # 1/100 of second
time = 5  # seconds
iterations = 5000
test_cases = 10

data = dict(c
    constraints=constraints,
    frequency=freq,
    time=time,
    iterations=iterations,
    test_cases=test_cases,
    length=(time/freq)*iterations,
    rnd_method="random",
    shuffle=False
)

cfg = Configurator("dnn_data_default")
cfg.configure(data=data)
cfg.write()

Deep Neural Net Default Configuration

In [3]:
columns_euler = list(MAXIMUMS_EULER.keys())

# Deep Neural Network Configuration
# Configuring DNN inputs
input_dict = {"Time": 10}
input_dict.update(MAXIMUMS_QUAT)

[*input_columns], [*input_norm_factor] = zip(*input_dict.items())
input_dim = len(input_columns)

# Configuring DNN outputs
output_dict = MAXIMUMS_QUAT
[*output_columns], [*output_norm_factor] = zip(*output_dict.items())
output_dim = len(output_columns)

network = dict(
    INPUT_FEATURES=input_columns,
    INPUT_FEATURES_EULER=["Time"] + columns_euler,
    INPUT_NORM_FACTORS=input_norm_factor,
    INPUT_DIM=input_dim,
    OUTPUT_FEATURES=output_columns,
    OUTPUT_FEATURES_EULER=columns_euler,
    OUTPUT_NORM_FACTORS=output_norm_factor,
    OUTPUT_DIM=output_dim,
    NETWORK_TYPE="DNN",
    optimizer="adam",
    metrics=["mse"],
)

dirs = dict(
    model_dir="models",
    checkpoints_dir="model_checkpoints",
    tuner_dir="tuner",
    train_dir="training_data",
    test_dir="test_data",
)

tuner = dict(
    activation_functions=["tanh", "sigmoid"],
    depth_range= dict(
        min=6,
        max=12,
        step=2),
    
    width_range=[20, 50, 2],
    regularization_range=[1e-11, 0.001224, "log"],
    kernel_initializer_choices=[
        "glorot_normal",
        "glorot_uniform",
        "he_normal",
        "he_uniform",
        "random_normal",
        "random_uniform",
        "zeros",
    ],
    bias_initializer_choices=["zeros", "random_uniform"],
    learning_rate_choices=[1e-2, 1e-3, 1e-4],
)

cfg = Configurator("dnn_default")
cfg.configure(network, dirs, tuner)
cfg.write()

Recurrent Neural Net Default Configuration

In [ ]:
columns_euler = list(MAXIMUMS_EULER.keys())

# Recurrent Neural Network Configuration
# Configuring RNN inputs
config_name = "rnn_default"
    
input_dict = MAXIMUMS_QUAT
[*input_columns], [*input_norm_factor] = zip(*input_dict.items())
input_dim = len(input_columns)

# Configuring RNN outputs
output_dict = MAXIMUMS_QUAT
[*output_columns], [*output_norm_factor] = zip(*output_dict.items())
output_dim = len(output_columns)


network = dict(
    INPUT_FEATURES=input_columns,
    INPUT_FEATURES_EULER=columns_euler,
    INPUT_NORM_FACTORS=input_norm_factor,
    INPUT_DIM=input_dim,
    OUTPUT_FEATURES=output_columns,
    OUTPUT_FEATURES_EULER=columns_euler,
    OUTPUT_NORM_FACTORS=output_norm_factor,
    OUTPUT_DIM=output_dim,
    NETWORK_TYPE="RNN",
    optimizer="adam",
    metrics=["mse"],
)

dirs = dict(
    model="models",
    checkpoints="model_checkpoints",
    tuner="tuner"
)

tuner = dict(
    type="hyperband",
    project_name = "RNN_standard"
    hyper_parameters=dict(
        activation_functions=["tanh", "sigmoid"],
        depth_range=[6, 12, 2],
        width_range=[20, 50, 2],
        regularization_range=[1e-11, 0.001224, "log"],
        kernel_initializer_choices=[
            "glorot_normal",
            "glorot_uniform",
            "he_normal",
            "he_uniform",
            "random_normal",
            "random_uniform",
            "zeros",
        ],
        bias_initializer_choices=["zeros", "random_uniform"],
        learning_rate_choices=[1e-2, 1e-3, 1e-4],
    )
)

cfg = Configurator("rnn_default")
cfg.configure(network, dirs, tuner)
cfg.write()